In [85]:
%load_ext dotenv
%dotenv ../../src/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [86]:
import sys
sys.path.append("../../src")

In [87]:
from utils.logger import get_logger
_logs = get_logger(__name__)
_logs.info("Notebook started successfully")

2025-11-14 14:46:55,621, 2927690305.py, 3, INFO, Notebook started successfully


In [88]:
import pandas as pd
import os
from glob import glob

In [89]:
import random

stock_files = glob(os.path.join(os.getenv('SRC_DIR'), "raw/*.csv"))


In [90]:
stock_files

['../../data/raw\\fundamentals.csv',
 '../../data/raw\\prices-split-adjusted.csv',
 '../../data/raw\\prices.csv',
 '../../data/raw\\securities.csv']

In [91]:
fundamentals = pd.read_csv(stock_files[0])
adj_prices = pd.read_csv(stock_files[1])
prices = pd.read_csv(stock_files[2])
securities = pd.read_csv(stock_files[3])

In [92]:
fundamentals.columns


Index(['Unnamed: 0', 'Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
       'Goodwill', 'Gross Margin', 'Gross Profit', 'Income Tax',
       'Intangible Assets', 'Interest Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term Debt', 'Long-Term Investments',
       'Minority Interest', 'Misc. Stocks', 'Net Borrowings', 'Net Cash Flow',
       'Net Cash Flow-Operating', 'Net Cash Flows-Financing',
       'Net Cash Flows-Investing', 'Net Income', 'Net Income Ad

In [93]:
prices.columns, adj_prices.columns

(Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object'),
 Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object'))

In [94]:
securities.columns

Index(['Ticker symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date first added',
       'CIK'],
      dtype='object')

In [95]:
# Standardize column names
for df in [fundamentals, prices, adj_prices, securities]:
    df.columns = df.columns.str.lower().str.strip()

In [96]:
fundamentals.columns

Index(['unnamed: 0', 'ticker symbol', 'period ending', 'accounts payable',
       'accounts receivable', 'add'l income/expense items', 'after tax roe',
       'capital expenditures', 'capital surplus', 'cash ratio',
       'cash and cash equivalents', 'changes in inventories', 'common stocks',
       'cost of revenue', 'current ratio', 'deferred asset charges',
       'deferred liability charges', 'depreciation',
       'earnings before interest and tax', 'earnings before tax',
       'effect of exchange rate',
       'equity earnings/loss unconsolidated subsidiary', 'fixed assets',
       'goodwill', 'gross margin', 'gross profit', 'income tax',
       'intangible assets', 'interest expense', 'inventory', 'investments',
       'liabilities', 'long-term debt', 'long-term investments',
       'minority interest', 'misc. stocks', 'net borrowings', 'net cash flow',
       'net cash flow-operating', 'net cash flows-financing',
       'net cash flows-investing', 'net income', 'net income ad

In [97]:
# Rename key columns for consistency
fundamentals = fundamentals.rename(columns={
    'ticker symbol': 'ticker',
    'period ending': 'date'
})
prices = prices.rename(columns={'symbol': 'ticker'})
adj_prices = adj_prices.rename(columns={'symbol': 'ticker'})
securities = securities.rename(columns={'ticker symbol': 'ticker'})

In [98]:
securities.columns

Index(['ticker', 'security', 'sec filings', 'gics sector', 'gics sub industry',
       'address of headquarters', 'date first added', 'cik'],
      dtype='object')

In [99]:
adj_prices.shape

(851264, 7)

In [100]:
fundamentals.shape

(1781, 79)

In [101]:
securities.shape

(505, 8)

In [102]:
# Unique tickers in each dataset
tickers_fund = set(fundamentals['ticker'].unique())
tickers_adj_price = set(adj_prices['ticker'].unique())
tickers_price = set(adj_prices['ticker'].unique())
tickers_sec = set(securities['ticker'].unique())

In [103]:
# --- Compare fundamentals vs adjusted prices ---
print("Tickers in fundamentals but not in prices:", tickers_fund - tickers_price)
print("Tickers in prices but not in fundamentals:", tickers_price - tickers_fund)

Tickers in fundamentals but not in prices: {'UA'}
Tickers in prices but not in fundamentals: {'GOOG', 'RF', 'SLB', 'PSA', 'ADP', 'URI', 'PRGO', 'IR', 'ICE', 'FTI', 'AET', 'CMCSA', 'FOXA', 'GS', 'ACN', 'AES', 'CBS', 'GOOGL', 'COP', 'PXD', 'NI', 'MNK', 'RIG', 'MDT', 'FITB', 'KHC', 'CA', 'NOC', 'LNC', 'RAI', 'DOW', 'JNJ', 'NRG', 'FOX', 'L', 'GE', 'PLD', 'DTE', 'AGN', 'FTV', 'A', 'ORCL', 'COH', 'EVHC', 'JCI', 'ESRX', 'TROW', 'ENDP', 'TWX', 'WBA', 'BLK', 'MSI', 'RTN', 'BEN'}


In [104]:
# --- Compare fundamentals vs securities ---
print("Tickers in fundamentals but not in securities:", tickers_fund - tickers_sec)
print("Tickers in securities but not in fundamentals:", len(tickers_sec - tickers_fund))


Tickers in fundamentals but not in securities: set()
Tickers in securities but not in fundamentals: 57


In [105]:
# --- Compare prices vs securities ---
print("Tickers in prices but not in securities:", tickers_price - tickers_sec)
print("Tickers in securities but not in prices:", tickers_sec - tickers_price)

Tickers in prices but not in securities: set()
Tickers in securities but not in prices: {'UA', 'MS', 'BRK.B', 'BF.B'}


In [106]:
# --- Compare prices vs adj_prices ---
print("Tickers in prices but not in adjusted prices:", tickers_price - tickers_adj_price)
print("Tickers in adjusted prices but not in prices:", tickers_adj_price - tickers_price)

Tickers in prices but not in adjusted prices: set()
Tickers in adjusted prices but not in prices: set()


In [107]:
common_tickers = tickers_fund & tickers_adj_price & tickers_sec
print("Number of tickers present in all three datasets:", len(common_tickers))

Number of tickers present in all three datasets: 447


In [108]:
# selecting tickers that are present in all the datasets
fundamentals = fundamentals[fundamentals['ticker'].isin(common_tickers)]
adj_prices = adj_prices[adj_prices['ticker'].isin(common_tickers)]
securities = securities[securities['ticker'].isin(common_tickers)]

In [109]:
# convert the date to datetime column
fundamentals['date'] = pd.to_datetime(fundamentals['date'])
adj_prices['date']   = pd.to_datetime(adj_prices['date'])


In [110]:

# Filter fundamentals *up to* the cutoff date
cutoff_date = pd.to_datetime("2016-01-01")
fundamentals_cut = fundamentals[fundamentals['date'] <= cutoff_date].copy()


In [111]:
fundamentals_cut.shape

(1556, 79)

- **This approiach only pick the last available fundamanetal data**

In [112]:
# # For each ticker, keep the latest available record
# fundamentals_latest = (
#     fundamentals_cut
#     .sort_values(['ticker', 'date'])
#     .groupby('ticker')
#     .tail(1)          # last record per ticker
#     .reset_index(drop=True)
# )

In [113]:
 # fundamentals_latest.shape

In [114]:
# fundamentals_latest.head()

- **This approach pivots the table to include all available fundamental data**

In [115]:
max_years = fundamentals_cut.groupby('ticker').size().max()
print(max_years)


4


In [116]:
fundamentals_cut = fundamentals_cut.sort_values(['ticker', 'date'])

fundamentals_cut['year_idx'] = fundamentals_cut.groupby('ticker').cumcount() + 1


In [117]:
fundamentals_cut.columns

Index(['unnamed: 0', 'ticker', 'date', 'accounts payable',
       'accounts receivable', 'add'l income/expense items', 'after tax roe',
       'capital expenditures', 'capital surplus', 'cash ratio',
       'cash and cash equivalents', 'changes in inventories', 'common stocks',
       'cost of revenue', 'current ratio', 'deferred asset charges',
       'deferred liability charges', 'depreciation',
       'earnings before interest and tax', 'earnings before tax',
       'effect of exchange rate',
       'equity earnings/loss unconsolidated subsidiary', 'fixed assets',
       'goodwill', 'gross margin', 'gross profit', 'income tax',
       'intangible assets', 'interest expense', 'inventory', 'investments',
       'liabilities', 'long-term debt', 'long-term investments',
       'minority interest', 'misc. stocks', 'net borrowings', 'net cash flow',
       'net cash flow-operating', 'net cash flows-financing',
       'net cash flows-investing', 'net income', 'net income adjustments',
    

In [118]:
exclude_cols = ['ticker', 'year_idx', 'date', 'unnamed: 0']  # add any other non-feature cols
feature_cols = [col for col in fundamentals_cut.columns if col not in exclude_cols]

fundamentals_wide = fundamentals_cut.pivot(
    index='ticker',
    columns='year_idx',
    values=feature_cols
)



In [119]:
fundamentals_wide.columns = [
    f"{feat}_{year}" for feat, year in fundamentals_wide.columns
]

fundamentals_wide = fundamentals_wide.reset_index()


In [120]:
fundamentals_wide.shape

(447, 305)

In [121]:
fundamentals_wide.head()

,ticker,accounts payable_1,accounts payable_2,accounts payable_3,accounts payable_4,accounts receivable_1,accounts receivable_2,accounts receivable_3,accounts receivable_4,add'l income/expense items_1,...,for year_3,for year_4,earnings per share_1,earnings per share_2,earnings per share_3,earnings per share_4,estimated shares outstanding_1,estimated shares outstanding_2,estimated shares outstanding_3,estimated shares outstanding_4
0,AAL,3.068000e+09,4.975000e+09,4.668000e+09,5.102000e+09,-2.220000e+08,-9.300000e+07,-1.600000e+08,3.520000e+08,-1.961000e+09,...,2014.0,2015.0,-5.60,-11.25,4.02,11.39,3.350000e+08,1.630222e+08,7.169154e+08,6.681299e+08
1,AAP,2.409453e+09,2.609239e+09,3.616038e+09,NaN,-8.948200e+07,-3.242800e+07,-4.820900e+07,NaN,6.000000e+05,...,2014.0,NaN,5.29,5.36,6.75,NaN,7.328355e+07,7.308918e+07,7.315926e+07,NaN
2,AAPL,3.622300e+10,4.864900e+10,6.067100e+10,NaN,-1.949000e+09,-6.452000e+09,-3.124000e+09,NaN,1.156000e+09,...,2015.0,NaN,40.03,6.49,9.28,NaN,9.252311e+08,6.087827e+09,5.753664e+09,NaN
3,ABBV,5.734000e+09,6.448000e+09,6.954000e+09,8.463000e+09,2.230000e+08,6.810000e+08,-1.720000e+08,-1.076000e+09,-8.000000e+06,...,2014.0,2015.0,NaN,2.58,1.11,3.15,NaN,1.600000e+09,1.598198e+09,1.633016e+09
4,ABC,1.487064e+10,1.725016e+10,2.157823e+10,NaN,-2.312518e+09,-9.382860e+08,-1.478793e+09,NaN,-4.400000e+04,...,2015.0,NaN,1.88,1.22,-0.62,NaN,2.306952e+08,2.247787e+08,2.228468e+08,NaN


In [122]:
fundamentals_latest=fundamentals_wide

In [123]:
fundamentals_latest['ticker'].value_counts()[fundamentals_latest['ticker'].value_counts()==1]

ticker
AAL     1
AAP     1
AAPL    1
ABBV    1
ABC     1
       ..
YHOO    1
YUM     1
ZBH     1
ZION    1
ZTS     1
Name: count, Length: 447, dtype: int64

In [124]:
securities['ticker'].value_counts()[securities['ticker'].value_counts()==1]

ticker
MMM     1
ABT     1
ABBV    1
ATVI    1
AYI     1
       ..
YHOO    1
YUM     1
ZBH     1
ZION    1
ZTS     1
Name: count, Length: 447, dtype: int64

In [162]:
fundamentals_latest['ticker'].size

447

In [163]:
securities['ticker'].size

447

- **We merge the fundamental and security tables**

In [125]:
fund_sec = pd.merge(fundamentals_latest, securities, on='ticker', how='left')

In [126]:
fund_sec.shape

(447, 312)

In [127]:
fund_sec.columns

Index(['ticker', 'accounts payable_1', 'accounts payable_2',
       'accounts payable_3', 'accounts payable_4', 'accounts receivable_1',
       'accounts receivable_2', 'accounts receivable_3',
       'accounts receivable_4', 'add'l income/expense items_1',
       ...
       'estimated shares outstanding_2', 'estimated shares outstanding_3',
       'estimated shares outstanding_4', 'security', 'sec filings',
       'gics sector', 'gics sub industry', 'address of headquarters',
       'date first added', 'cik'],
      dtype='object', length=312)

In [128]:
# keep_cols = [
#     # Identity
#     'ticker', 'gics sector',

#     # Financial Ratios
#     'current ratio', 'quick ratio', 'cash ratio', 
#     'gross margin', 'operating margin', 'profit margin',
#     'pre-tax margin', 'pre-tax roe', 'after tax roe',
#     'earnings per share',

#     # Balance Sheet (high-level)
#     'total assets', 'total current assets', 'total current liabilities',
#     'total liabilities', 'total equity', 'long-term debt',
#     'inventory', 'investments', 'cash and cash equivalents',
#     'fixed assets', 'intangible assets', 'goodwill',

#     # Cash Flow
#     'net cash flow-operating', 'net cash flows-investing',
#     'net cash flows-financing', 'capital expenditures',

#     # Income Statement
#     'total revenue', 'gross profit', 'operating income',
#     'earnings before tax', 'net income'
# ]


In [129]:
keep_cols=fund_sec.columns

In [130]:
fund_sec.columns = fund_sec.columns.str.lower().str.strip()

In [131]:
# Filter only the columns that exist (avoids KeyErrors)
keep_cols_final = [col for col in keep_cols if col in fund_sec.columns]
print("Keeping", len(keep_cols_final), "columns out of", len(fund_sec.columns))

Keeping 312 columns out of 312


In [132]:
fund_sec_clean = fund_sec[keep_cols_final].copy()

In [133]:
print(f"Original shape: {fund_sec.shape}")
print(f"Cleaned shape:  {fund_sec_clean.shape}")
fund_sec_clean.head()

Original shape: (447, 312)
Cleaned shape:  (447, 312)


,ticker,accounts payable_1,accounts payable_2,accounts payable_3,accounts payable_4,accounts receivable_1,accounts receivable_2,accounts receivable_3,accounts receivable_4,add'l income/expense items_1,...,estimated shares outstanding_2,estimated shares outstanding_3,estimated shares outstanding_4,security,sec filings,gics sector,gics sub industry,address of headquarters,date first added,cik
0,AAL,3.068000e+09,4.975000e+09,4.668000e+09,5.102000e+09,-2.220000e+08,-9.300000e+07,-1.600000e+08,3.520000e+08,-1.961000e+09,...,1.630222e+08,7.169154e+08,6.681299e+08,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
1,AAP,2.409453e+09,2.609239e+09,3.616038e+09,NaN,-8.948200e+07,-3.242800e+07,-4.820900e+07,NaN,6.000000e+05,...,7.308918e+07,7.315926e+07,NaN,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",2015-07-09,1158449
2,AAPL,3.622300e+10,4.864900e+10,6.067100e+10,NaN,-1.949000e+09,-6.452000e+09,-3.124000e+09,NaN,1.156000e+09,...,6.087827e+09,5.753664e+09,NaN,Apple Inc.,reports,Information Technology,Computer Hardware,"Cupertino, California",1982-11-30,320193
3,ABBV,5.734000e+09,6.448000e+09,6.954000e+09,8.463000e+09,2.230000e+08,6.810000e+08,-1.720000e+08,-1.076000e+09,-8.000000e+06,...,1.600000e+09,1.598198e+09,1.633016e+09,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
4,ABC,1.487064e+10,1.725016e+10,2.157823e+10,NaN,-2.312518e+09,-9.382860e+08,-1.478793e+09,NaN,-4.400000e+04,...,2.247787e+08,2.228468e+08,NaN,AmerisourceBergen Corp,reports,Health Care,Health Care Distributors,"Chesterbrook, Pennsylvania",NaN,1140859


In [134]:
# Change the name of price data as it is processed
time_series_dataset = adj_prices

In [135]:
time_series_dataset = time_series_dataset.sort_values(['ticker', 'date']).reset_index(drop=True)

In [136]:
time_series_dataset.shape

(760913, 7)

In [137]:
# Daily returns %
time_series_dataset['return_pct'] = time_series_dataset.groupby('ticker')['close'].pct_change() * 100


In [138]:
# Rolling volatility
time_series_dataset['vol_5d'] = time_series_dataset.groupby('ticker')['return_pct'].rolling(5).std().reset_index(0, drop=True)
time_series_dataset['vol_10d'] = time_series_dataset.groupby('ticker')['return_pct'].rolling(10).std().reset_index(0, drop=True)

In [139]:
# Momentum
time_series_dataset['momentum_5d'] = time_series_dataset.groupby('ticker')['close'].diff(5)
time_series_dataset['momentum_10d'] = time_series_dataset.groupby('ticker')['close'].diff(10)

In [140]:
# Moving averages
time_series_dataset['sma_5d'] = time_series_dataset.groupby('ticker')['close'].rolling(5).mean().reset_index(0, drop=True)
time_series_dataset['sma_10d'] = time_series_dataset.groupby('ticker')['close'].rolling(10).mean().reset_index(0, drop=True)


In [141]:
# High-low range %
time_series_dataset['hl_range_pct'] = (time_series_dataset['high'] - time_series_dataset['low']) / time_series_dataset['close'] * 100

In [142]:
# Open-close change %
time_series_dataset['oc_change_pct'] = (time_series_dataset['close'] - time_series_dataset['open']) / time_series_dataset['open'] * 100

In [143]:
# Rolling average volume
time_series_dataset['avg_volume_5d'] = time_series_dataset.groupby('ticker')['volume'].rolling(5).mean().reset_index(0, drop=True)
time_series_dataset['avg_volume_10d'] = time_series_dataset.groupby('ticker')['volume'].rolling(10).mean().reset_index(0, drop=True)

In [144]:
# Fill NaNs caused by rolling windows
time_series_dataset.fillna(0, inplace=True)

In [145]:
time_series_dataset.columns

Index(['date', 'ticker', 'open', 'close', 'low', 'high', 'volume',
       'return_pct', 'vol_5d', 'vol_10d', 'momentum_5d', 'momentum_10d',
       'sma_5d', 'sma_10d', 'hl_range_pct', 'oc_change_pct', 'avg_volume_5d',
       'avg_volume_10d'],
      dtype='object')

In [146]:
# Filter up to cut-off date to avoid leakage
ts_filtered = time_series_dataset[time_series_dataset['date'] <= cutoff_date]

In [147]:
# Aggregate per ticker
agg_features = ts_filtered.groupby('ticker').agg(
    # Price stats
    last_close=('close', 'last'),
    max_close=('close', 'max'),
    min_close=('close', 'min'),

    # Returns
    avg_return=('return_pct', 'mean'),
    std_return=('return_pct', 'std'),

    # Volatility
    avg_vol_5d=('vol_5d', 'mean'),
    avg_vol_10d=('vol_10d', 'mean'),

    # Momentum
    last_momentum_5d=('momentum_5d', 'last'),
    last_momentum_10d=('momentum_10d', 'last'),

    # Moving averages
    last_sma_5d=('sma_5d', 'last'),
    last_sma_10d=('sma_10d', 'last'),

    # Price range / price change
    avg_hl_range_pct=('hl_range_pct', 'mean'),
    avg_oc_change_pct=('oc_change_pct', 'mean'),

    # Volume
    last_avg_volume_5d=('avg_volume_5d', 'last'),
    last_avg_volume_10d=('avg_volume_10d', 'last')
).reset_index()

In [148]:
# Merge aggregated features into ticker-level dataset
ticker_features = pd.merge(fund_sec_clean, agg_features, on='ticker', how='left')

In [149]:
print(" Ticker-level dataset enriched with aggregated time-series features.")
print("Shape:", ticker_features.shape)

 Ticker-level dataset enriched with aggregated time-series features.
Shape: (447, 327)


In [150]:
missing_summary = ticker_features.isnull().sum()
print(missing_summary[missing_summary > 0])

accounts payable_2         2
accounts payable_3        12
accounts payable_4       218
accounts receivable_2      2
accounts receivable_3     12
                        ... 
last_sma_10d               1
avg_hl_range_pct           1
avg_oc_change_pct          1
last_avg_volume_5d         1
last_avg_volume_10d        1
Length: 250, dtype: int64


In [151]:
numeric_cols = ticker_features.select_dtypes(include='number').columns

In [152]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
ticker_features[numeric_cols] = imputer.fit_transform(ticker_features[numeric_cols])


In [153]:
# Identify categorical columns, **excluding 'ticker'**
categorical_cols = ticker_features.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != 'ticker']

In [154]:
categorical_cols

['security',
 'sec filings',
 'gics sector',
 'gics sub industry',
 'address of headquarters',
 'date first added']

In [155]:
ticker_features[categorical_cols].nunique()

security                   447
sec filings                  1
gics sector                 11
gics sub industry          121
address of headquarters    246
date first added           228
dtype: int64

In [156]:
# Impute missing values with most frequent category
for col in categorical_cols:
    ticker_features[col] = ticker_features[col].fillna(ticker_features[col].mode().iloc[0])

In [157]:
PROCESSED_DIR = os.path.join(os.getenv('SRC_DIR'), "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [158]:
time_series_path = os.path.join(PROCESSED_DIR, "02_time_series_dataset.csv")
ticker_features_path = os.path.join(PROCESSED_DIR, "01_ticker_features.csv")

In [159]:
time_series_dataset.to_csv(time_series_path, index=False)
ticker_features.to_csv(ticker_features_path, index=False)